In [ ]:
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from baseline_embedding import get_dataset
import os
import time
import torch.nn as nn
import numpy as np
import gc
from sklearn.metrics import cohen_kappa_score, accuracy_score

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-07-22 18:22:12.703219: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-22 18:22:12.710683: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753208532.719806   58981 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753208532.722507   58981 cuda_blas.cc:1407] Unable to register 

In [2]:
from config import config

args = config['nikl']

In [3]:
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

In [4]:
def get_embedded_essay(essays, is_train=True):        
    embedded_essay_raw = pd.read_csv(os.path.join(args['emb_file_path'], f"{args['train_dataset_path'].split('/')[1]}_{'train' if is_train else 'valid'}_{'notlabeled' if args['is_topic_label'] == False else 'labeled'}.csv"), encoding='cp949')
    print(embedded_essay_raw.shape)
    embedded_essay = []
    tmp_ix = 0
    for ix, essay_raw in enumerate(essays):
        tmp_len = len(essay_raw)
        essay = embedded_essay_raw[tmp_ix:tmp_ix + tmp_len]
        embedded_essay.append(essay)
        tmp_ix += tmp_len
    return embedded_essay

In [5]:
def compute_metrics(y_sent_pred, y_test):
    metrics = {}
    all_kappas = []
    for i in range(len(args['rubric'])):
        metrics[args['rubric'][i]] = {}
        y_pred = y_sent_pred[:, i]
        y_true = y_test[:, i]
        accuracy = accuracy_score(y_true, y_pred)
        kappa = cohen_kappa_score(y_true, y_pred, weights='quadratic')
        metrics[args['rubric'][i]]['accuracy'] = accuracy
        metrics[args['rubric'][i]]['kappa'] = kappa
        all_kappas.append(kappa)

    metrics['mean'] = {}
    overall_accuracy = accuracy_score(y_test.flatten(), y_sent_pred.flatten())
    overall_kappa = np.mean(all_kappas)
    metrics['mean']['accuracy'] = overall_accuracy
    metrics['mean']['kappa'] = overall_kappa

    metrics['overall'] = {}
    metrics['overall']['kappa'] = cohen_kappa_score(y_test.flatten(), y_sent_pred.flatten(), weights='quadratic')
    return metrics

In [6]:
class GRUScoreModule(nn.Module):
    def __init__(self,output_dim,hidden_dim, dropout=0.5):
        super(GRUScoreModule, self).__init__()
        self.gru = nn.GRU(768,hidden_dim, dropout=dropout, batch_first=True, bidirectional=True)        
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim*2, output_dim)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x, _ = self.gru(x)
        
        x = x[:, -1, :]  # Use the output of the last time step
        x = self.dropout(x)
        x = self.fc(x)
        x = self.sigmoid(x)
        return x

In [7]:
class EssayDataset(Dataset):
    def __init__(self, embedded_essays, labels):
        # 원본 데이터를 그대로 저장 (패딩하지 않음)
        self.embedded_essays = embedded_essays
        # labels만 텐서로 변환
        self.labels = torch.tensor(labels, dtype=torch.float32)
        self.maxlen = 128

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # 필요할 때마다 패딩 및 텐서 변환 수행
        essay = self.embedded_essays[idx]
        
        # numpy array로 변환 (DataFrame인 경우)
        if hasattr(essay, 'values'):
            essay = essay.values
        
        # 패딩 수행
        if len(essay) < self.maxlen:
            # pre-padding (앞쪽에 0 추가)
            padded = np.zeros((self.maxlen, essay.shape[1]), dtype=np.float32)
            padded[-len(essay):] = essay
        else:
            # maxlen까지만 자르기
            padded = essay[:self.maxlen].astype(np.float32)
        
        # 텐서로 변환
        padded_tensor = torch.tensor(padded, dtype=torch.float32)
        
        return padded_tensor, self.labels[idx]


In [8]:
train_essay, valid_essay, train_y, valid_y = get_dataset()
train_embedded_essay = get_embedded_essay(train_essay, is_train=True)
valid_embedded_essay = get_embedded_essay(valid_essay, is_train=False)

(194416, 768)
(22267, 768)


In [9]:
train_dataset = EssayDataset(train_embedded_essay, train_y)
valid_dataset = EssayDataset(valid_embedded_essay, valid_y) 

In [10]:
valid_y

array([[0.6, 0.5, 0.5, ..., 0.6, 0.8, 0.9],
       [1. , 1. , 1. , ..., 0.8, 0.9, 0.9],
       [0.9, 1. , 0.7, ..., 0.6, 0.7, 0.9],
       ...,
       [0.7, 1. , 0.9, ..., 0.8, 0.9, 0.9],
       [0.8, 0.7, 0.7, ..., 0.7, 0.8, 0.9],
       [0.4, 0.8, 0.5, ..., 0.4, 0.6, 0.8]])

In [11]:
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

In [12]:
import torch.optim as optim
n_outputs = len(args['rubric'])

dropout = 0.5
learning_rate = 0.001
n_epochs = 100

model = GRUScoreModule(output_dim=n_outputs,hidden_dim=128, dropout=dropout).cuda()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn(


In [13]:
patience = 10
train_loss_list = []
val_loss_list = []
best_val_loss = float('inf')
early_stopping_counter = 0
prev_time = time.time()
set_seed(42)
for epoch in range(n_epochs):
    model.train()
    train_loss = 0
    for inputs,labels in train_loader:
        inputs ,labels = inputs.cuda(), labels.cuda()
        optimizer.zero_grad()
        outputs = model(inputs)        
        loss = criterion(outputs, labels)
            
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    model.eval()
    all_outputs = []
    val_loss = 0
    with torch.no_grad():
        for inputs, labels in valid_loader:
            inputs, labels = inputs.cuda(),labels.cuda()
            outputs = model(inputs)            
            loss = criterion(outputs, labels)
            all_outputs.extend(outputs.cpu().numpy())
            val_loss += loss.item()

    train_loss /= len(train_loader)
    val_loss /= len(valid_loader)
    train_loss_list.append(train_loss)
    val_loss_list.append(val_loss)
    print(f'Epoch {epoch+1}/{n_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Time Elapsed: {time.time() - prev_time:.4f}')
    prev_time = time.time()
    
    if val_loss < best_val_loss:
        best_outputs = np.array(all_outputs)
        if not os.path.exists('./model'):
            os.makedirs('./model')
        torch.save(model.state_dict(), './model/kobert_model.pth')
        best_val_loss = val_loss
        early_stopping_counter = 0
    else:
        early_stopping_counter += 1
        if early_stopping_counter >= patience:
            print("Early stopping")
            break

Epoch 1/100, Train Loss: 0.0280, Val Loss: 0.0245, Time Elapsed: 2.2111
Epoch 2/100, Train Loss: 0.0235, Val Loss: 0.0234, Time Elapsed: 2.0267
Epoch 3/100, Train Loss: 0.0222, Val Loss: 0.0222, Time Elapsed: 2.2007
Epoch 4/100, Train Loss: 0.0214, Val Loss: 0.0212, Time Elapsed: 2.3027
Epoch 5/100, Train Loss: 0.0208, Val Loss: 0.0209, Time Elapsed: 2.4139
Epoch 6/100, Train Loss: 0.0204, Val Loss: 0.0204, Time Elapsed: 2.5016
Epoch 7/100, Train Loss: 0.0199, Val Loss: 0.0204, Time Elapsed: 2.5967
Epoch 8/100, Train Loss: 0.0199, Val Loss: 0.0201, Time Elapsed: 2.6769
Epoch 9/100, Train Loss: 0.0191, Val Loss: 0.0202, Time Elapsed: 2.7146
Epoch 10/100, Train Loss: 0.0188, Val Loss: 0.0203, Time Elapsed: 2.7203
Epoch 11/100, Train Loss: 0.0182, Val Loss: 0.0202, Time Elapsed: 2.6861
Epoch 12/100, Train Loss: 0.0181, Val Loss: 0.0200, Time Elapsed: 2.7362
Epoch 13/100, Train Loss: 0.0176, Val Loss: 0.0198, Time Elapsed: 2.7450
Epoch 14/100, Train Loss: 0.0173, Val Loss: 0.0202, Time Ela

In [14]:
y_pred = best_outputs*5
y_test = np.array(valid_y)*5

In [15]:
pred = np.round(y_pred*2)/2
pred = np.clip(pred, 1 , 5)
pred = pred*2-1
real = y_test*2-1

In [16]:
metrics = compute_metrics(pred,real)
metrics

{'con1': {'accuracy': 0.26307053941908715, 'kappa': 0.4542025509728662},
 'con2': {'accuracy': 0.29128630705394193, 'kappa': 0.413018728302731},
 'con3': {'accuracy': 0.26639004149377593, 'kappa': 0.39591341644255507},
 'con4': {'accuracy': 0.2846473029045643, 'kappa': 0.4612244318356269},
 'con5': {'accuracy': 0.24066390041493776, 'kappa': 0.4237863476756063},
 'org1': {'accuracy': 0.23402489626556017, 'kappa': 0.4440012063813844},
 'org2': {'accuracy': 0.2854771784232365, 'kappa': 0.436483048240763},
 'exp1': {'accuracy': 0.3269709543568465, 'kappa': 0.4291323519448814},
 'exp2': {'accuracy': 0.30373443983402487, 'kappa': 0.502120365364841},
 'mean': {'accuracy': 0.2773628400184417, 'kappa': 0.43998693857347276},
 'overall': {'kappa': 0.5465917653248737}}